In [1]:
import sys
import altair as alt
import networkx as nx
import numpy as np
import pandas as pd
from faker import Faker

".." not in sys.path and sys.path.append("..")
from alph import alph, layers, layout, util

util.set_altair_renderer()

In [3]:
G = nx.read_gml("data/lesmis.gml")
print(f"nodes: {len(G.nodes())}, links: {len(G.edges())}")

nodes: 77, links: 254


In [80]:
communities = nx.algorithms.community.louvain_communities(G, weight="value", resolution=1, seed=seed)
node_to_group = {n: c for c, ns in enumerate(communities) for n in ns}

nx.set_node_attributes(G, {n: n for n in G.nodes()}, "name")
nx.set_node_attributes(G, node_to_group, "group")

print(list(node_to_group.items())[:3])

[('Champtercier', 0), ('Napoleon', 0), ('CountessDeLo', 0)]


In [49]:
seed = 42
max_edge_value = max([d[2] for d in G.edges(data="value")])

alph(
    G,
    weight_attr="value",
    layout_fn=lambda g: layout.force_atlas(
        g, gravity=8, outboundAttractionDistribution=False,  iterations=1000, seed=seed
    ),
    node_args=dict(
        size=alt.value(6**2),
        fill=alt.Color("group", scale=alt.Scale(scheme="category10"), legend=None),
        tooltip_attrs=["name", "group"]
    ),
    edge_args=dict(
        color="#333",
        opacity=alt.Size(
            "value:Q",
            scale=alt.Scale(domain=[0,max_edge_value], range=[0.5, 0.8]),
            legend=None
        ),
        strokeWidth=alt.Size(
            "value:Q",
            scale=alt.Scale(domain=[0,max_edge_value], range=[0.1, 1]),
            legend=None
        ),
    ),
).configure_view(strokeWidth=0).properties(width=600, height=420)

alt.LayerChart(...)

In [131]:
nx.set_edge_attributes(G, {(u,v): w for u, v, w in G.edges(data="value")}, "weight")
groups = list(set([x[1] for x in G.nodes(data="group")]))

def combo_chart(layout_fn):
    return alph(
        G,
        weight_attr="weight",
        layout_fn=layout_fn,
        node_args=dict(
            size=5**2,
            fill="#888",
            stroke="#222",
            strokeWidth=1,
            tooltip_attrs=["name", "group"],
        ),
        edge_args=dict(
            color="#333",
            opacity=alt.Size(
                "weight:Q",
                scale=alt.Scale(range=[0.5, 0.8]),
                legend=None
            ),
            strokeWidth=alt.Size(
                "weight:Q",
                scale=alt.Scale(range=[0.4, 1]),
                legend=None
            ),
        ),
        combo_group_by="group",
        combo_layout_fn=lambda g: layout.force_atlas(
            g,
            weight_attr="weight",
            strongGravityMode=True,
            gravity=6,
            edgeWeightInfluence=1.2,
            seed=seed,
        ),
        combo_node_additional_attrs={
            group: {"group": group or ""} for group in groups
        },
        combo_node_args=dict(
            fill="#fff",
            tooltip_attrs=["group:N"],
            label_attr="group:N",
            label_offset=12,
        ),
        combo_edge_args=dict(
            strokeWidth=alt.Size("weight", scale=alt.Scale(range=[0.5, 3]), legend=None),
        ),
        combo_size_scale_range=[40**2,100**2],
        combo_inner_graph_scale_factor=0.6,
        combo_empty_attr_action="promote",
        width=600,
        height=480,
    ).configure_view(strokeWidth=0)

In [133]:
combo_chart(
    layout_fn=lambda g: layout.force_atlas(
        g,
        weight_attr="weight",
        strongGravityMode=True,
        outboundAttractionDistribution=False,
        gravity=5,
        # edgeWeightInfluence=1,
        seed=seed,
    ),
)

alt.LayerChart(...)

In [132]:
combo_chart(layout_fn=nx.circular_layout)

alt.LayerChart(...)